In [1]:
!pip uninstall -y torch
!pip uninstall -y vllm

# !pip install -q torch==2.6.0
!pip install -q torch==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q vllm==0.8.5.post1


Found existing installation: torch 2.1.0
Uninstalling torch-2.1.0:
  Successfully uninstalled torch-2.1.0


In [1]:
import torch
print(torch.__version__)

import vllm
print(vllm.__version__)

# 2.6.0+cu124
# 0.8.5.post1

2.6.0+cu124


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 06-03 07:58:11 [__init__.py:239] Automatically detected platform cuda.


2025-06-03 07:58:13,279	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


0.8.5.post1


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
4


In [4]:
!pip install -U "pyzmq<26"

  Obtaining dependency information for pyzmq<26 from https://files.pythonhosted.org/packages/67/bf/6bc0977acd934b66eacab79cec303ecf08ae4a6150d57c628aa919615488/pyzmq-25.1.2-cp310-cp310-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 26.2.0
    Uninstalling pyzmq-26.2.0:
      Successfully uninstalled pyzmq-26.2.0


In [3]:
import psutil, torch, textwrap

gib = lambda b: b / 1024**3

cpu = psutil.virtual_memory()
print(textwrap.dedent(f"""
CPU RAM
  Total     : {gib(cpu.total):5.1f} GiB
  Available : {gib(cpu.available):5.1f} GiB
  Used      : {gib(cpu.used):5.1f} GiB ({cpu.percent:.1f}%)
"""))

print("GPU VRAM")
for i in range(torch.cuda.device_count()):
    free, total = map(gib, torch.cuda.mem_get_info(i))
    props = torch.cuda.get_device_properties(i)
    gpu_tot = gib(props.total_memory)
    gpu_used = total - free
    print(textwrap.dedent(f"""\
    GPU {i} - {props.name}
      Total     : {gpu_tot:5.1f} GiB
      Free      : {free:5.1f} GiB
      Used      : {gpu_used:5.1f} GiB ({gpu_used/gpu_tot*100:.1f}%)
    """))



CPU RAM
  Total     : 251.6 GiB
  Available : 233.4 GiB
  Used      :  16.3 GiB (7.2%)

GPU VRAM
GPU 0 - NVIDIA RTX A6000
  Total     :  47.5 GiB
  Free      :  46.5 GiB
  Used      :   1.1 GiB (2.3%)

GPU 1 - NVIDIA RTX A6000
  Total     :  47.5 GiB
  Free      :  47.3 GiB
  Used      :   0.3 GiB (0.5%)

GPU 2 - NVIDIA RTX A6000
  Total     :  47.5 GiB
  Free      :  47.3 GiB
  Used      :   0.3 GiB (0.5%)

GPU 3 - NVIDIA RTX A6000
  Total     :  47.5 GiB
  Free      :  47.3 GiB
  Used      :   0.3 GiB (0.5%)



In [7]:
!pip install -q datasets

In [8]:
from datasets import load_dataset
import pandas as pd
import sqlite3
import os

DB_FILE = 'dataset_4qwen3.db'

if os.path.exists(DB_FILE):
    print(f"'{DB_FILE}' already exists")
else:
    # Define columns including the new 'trace' column
    columns = ["question", "answer", "trace"]
    df = pd.DataFrame(columns=columns)

    # Load and process s1K-1.1 dataset
    s1 = load_dataset("simplescaling/s1K-1.1", split="train")
    df_s1 = pd.DataFrame(s1)[["question", "solution"]]
    df_s1.columns = ["question", "answer"]
    df_s1["trace"] = ""  # Add empty trace column
    df = pd.concat([df, df_s1], ignore_index=True)

    # Load and process LIMO dataset
    limo = load_dataset("GAIR/LIMO", split="train")
    df_limo = pd.DataFrame(limo)[["question", "answer"]]
    df_limo["trace"] = ""  # Add empty trace column
    df = pd.concat([df, df_limo], ignore_index=True)

    # Save as SQLite database
    conn = sqlite3.connect(DB_FILE)
    df.to_sql('dataset', conn, if_exists='replace', index=False)
    conn.close()
    print(f"Dataset saved as {DB_FILE}")


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/data/huggingface_cache/datasets--simplescaling--s1K-1.1/.no_exist/96c411f1fe4c49d20f0e2a1565f61e1a28b0b84d/s1K-1.1.py'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/data/huggingface_cache/datasets--simplescaling--s1K-1.1/.no_exist/96c411f1fe4c49d20f0e2a1565f61e1a28b0b84d/.huggingface.yaml'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/data/huggingface_cache/datasets--simplescaling--s1K-1.1/.no_exist/96c411f1fe4c49d20f0e2a1565f61e1a28b0b84d/dataset_infos.json'
Generating train split: 100%|██████████| 817/817 [00:00<00:00, 5249.59 examples/s]


Dataset saved as dataset_4qwen3.db


In [3]:
import pandas as pd
import sqlite3

DB_FILE = 'dataset_4qwen3.db'

# Read the SQLite database back into a DataFrame
conn = sqlite3.connect(DB_FILE)
df = pd.read_sql('SELECT * FROM dataset', conn)
conn.close()

print(f"Loaded {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

null_count = df['trace'].isnull().sum()
empty_string_count = (df['trace'] == '').sum()
whitespace_only_count = df['trace'].str.strip().eq('').sum() if df['trace'].dtype == 'object' else 0
total_empty = df['trace'].isnull().sum() + (df['trace'].str.strip() == '').sum()

print(f"Null/NaN values: {null_count}")
print(f"Empty strings: {empty_string_count}")
print(f"Whitespace-only strings: {whitespace_only_count}")
print(f"Total empty rows: {total_empty}")

df.head()

Loaded 1817 rows
Columns: ['question', 'answer', 'trace']
Null/NaN values: 0
Empty strings: 1817
Whitespace-only strings: 1817
Total empty rows: 1817


,question,answer,trace
0,"Given a rational number, write it as a fractio...",128,
1,Let $ \mathcal{H}$ be an infinite-dimensiona...,1. **Consider a countable subset \( S_0 \subse...,
2,Find the remainder when $9 \times 99 \times 99...,109,
3,Compute the mean molecular speed v in the heav...,167.0,
4,Two capacitors with capacitance values $C_{1}=...,1.3,


In [4]:
LARGE_GPU_INDEX = "0"
SMALL_GPU_INDEX = "3"

In [5]:
# Reverse Speculative Decoding with vLLM

# large model proposes, small model filters
# accept if large model token is within small model top 20 and has prob over 0.01
# vocab match mask also
# multiple trials on large model proposals
# live per-token stat
# html heatmap
# closely resembling HF version

# --------------------------- imports ---------------------------------------
import os, html, uuid, asyncio, contextlib, nest_asyncio, logging
from IPython.display import HTML, display

import torch
from huggingface_hub import snapshot_download
from vllm.engine.async_llm_engine import AsyncLLMEngine
from vllm.engine.arg_utils import AsyncEngineArgs
from vllm.sampling_params import SamplingParams, RequestOutputKind

nest_asyncio.apply()
torch.set_grad_enabled(False)
logging.disable(logging.INFO)

# --------------------------- configuration ---------------------------------
LARGE_MODEL_NAME  = "simplescaling/s1.1-7B"
SMALL_MODEL_NAME  = "Qwen/Qwen3-0.6B"

LARGE_TEMPERATURE = 0.7
SMALL_TEMPERATURE = 0.7
MAX_SEQ_LEN       = 8192
MAX_NEW_TOKENS    = MAX_SEQ_LEN - 1024

# ---------------- utility: temporarily set visible GPUs --------------------
@contextlib.contextmanager
def visible_gpus(devices: str):
    original = os.environ.get("CUDA_VISIBLE_DEVICES", "")
    os.environ["CUDA_VISIBLE_DEVICES"] = devices
    print(f"\nCUDA_VISIBLE_DEVICES = {devices}")
    try:
        yield
    finally:
        os.environ["CUDA_VISIBLE_DEVICES"] = original

# --------------------------- engine setup ----------------------------------
async def setup_engines():
    global large_engine, small_engine, large_tokenizer, small_tokenizer
    global large_vocab_size, small_vocab_size, vocab_match_mask
    
    large_checkpoint = snapshot_download(LARGE_MODEL_NAME)
    small_checkpoint = snapshot_download(SMALL_MODEL_NAME)

    with visible_gpus(LARGE_GPU_INDEX):
        print("torch sees", torch.cuda.device_count(), "GPU(s)")              
        large_engine = AsyncLLMEngine.from_engine_args(
            AsyncEngineArgs(model=large_checkpoint, 
                            tensor_parallel_size=1,
                            max_model_len=MAX_SEQ_LEN, 
                            gpu_memory_utilization=0.60,
                            dtype="float16"),
            start_engine_loop=True)
        
        large_tokenizer = await large_engine.get_tokenizer()

    with visible_gpus(SMALL_GPU_INDEX):
        print("torch sees", torch.cuda.device_count(), "GPU(s)")              
        small_engine = AsyncLLMEngine.from_engine_args(
            AsyncEngineArgs(model=small_checkpoint, 
                            tensor_parallel_size=1,
                            max_model_len=MAX_SEQ_LEN, 
                            gpu_memory_utilization=0.20,
                            dtype="float16"),
            start_engine_loop=True)
        
        small_tokenizer = await small_engine.get_tokenizer()

    # Get model configs using async methods
    large_model_config = await large_engine.get_model_config()
    small_model_config = await small_engine.get_model_config()
    
    large_vocab_size = large_model_config.get_vocab_size()
    small_vocab_size = small_model_config.get_vocab_size()
    
    print(f"Large vocab size: {large_vocab_size}")
    print(f"Small vocab size: {small_vocab_size}")
    print(f"Difference      : {abs(large_vocab_size - small_vocab_size)}")

    vocab_match_mask = torch.zeros(max(large_vocab_size, small_vocab_size), dtype=torch.float32)
    mismatches = []

    for idx in range(min(large_vocab_size, small_vocab_size)):
        large_token = large_tokenizer.convert_ids_to_tokens(idx)
        small_token = small_tokenizer.convert_ids_to_tokens(idx)
        
        if large_token == small_token:
            vocab_match_mask[idx] = 1.0
        else:
            mismatches.append((idx, large_token, small_token))

    print(f"Unmatched tokens: {len(mismatches)}")

# --------------------------- sampling params -------------------------------
large_sampling_params = SamplingParams(
    max_tokens  = 1,
    temperature = LARGE_TEMPERATURE,
    top_p       = 0.95, 
    logprobs    = 20,
    output_kind = RequestOutputKind.DELTA,
)
small_sampling_params = SamplingParams(
    max_tokens  = 1,
    temperature = SMALL_TEMPERATURE,
    top_p       = 0.95, 
    logprobs    = 20,
    output_kind = RequestOutputKind.DELTA,
)

# -------------------------- helper functions -------------------------------
async def one_step(engine, sampling_params, context):
    generator = engine.generate(context, sampling_params, request_id=str(uuid.uuid4()))
    return (await anext(generator)).outputs[0]

def html_heat(records):
    probability_min, probability_max = 0.0, 0.2
    def colour(probability):
        if probability >= probability_max: 
            return "rgb(0,0,0)"
        red = int(255 * (probability_max - probability) / (probability_max - probability_min))
        return f"rgb({red},0,0)"
    spans = []
    for record in records:
        text = html.escape(record['text']).replace(" ", "&nbsp;")
        style = f"color:{colour(record['small_probability'])};"
        if record['fallback']: 
            style += " text-decoration:underline;"
        spans.append(f"<span style='{style}'>{text}</span>")
    return HTML("<pre style='white-space:pre-wrap; line-height:1.45; "
                "font-family:inherit; background:#fff; padding:8px; "
                "border:1px solid #ddd;'>" + "".join(spans) + "</pre>")

# ------------------------- core decode loop --------------------------------
async def mixed_decode(prompt: str, max_new_tokens: int = MAX_NEW_TOKENS):
    context = prompt
    step_index = 0
    PROB_THRESHOLD = 0.01  
    NUM_TRIALS = 5 
    
    for _ in range(max_new_tokens):
        large_output, small_output = await asyncio.gather(
            one_step(large_engine, large_sampling_params, context),
            one_step(small_engine, small_sampling_params, context))

        if step_index < 3:
            print(f"  large_output: {large_output}")
            print(f"  small_output: {small_output}")

        # Extract probabilities from large model output - logprobs is a list
        large_logprobs_dict = large_output.logprobs[0]  
        large_probs = {}
        for token_id, logprob in large_logprobs_dict.items():
            if vocab_match_mask[token_id] > 0:  # Only include vocab-matched tokens
                large_probs[token_id] = torch.exp(torch.tensor(logprob.logprob)).item()  # Access .logprob attribute
        
        idx_pool = torch.tensor(list(large_probs.keys()))
        prob_pool = torch.tensor(list(large_probs.values()))
        prob_pool = prob_pool / prob_pool.sum()  # Normalize for sampling

        # Extract probabilities from small model output - compact dict for lookup  
        small_logprobs_dict = small_output.logprobs[0] 
        small_probs = {}
        for token_id, logprob in small_logprobs_dict.items():
            small_probs[token_id] = torch.exp(torch.tensor(logprob.logprob)).item()  # Access .logprob attribute

        # Try to accept a token from large model's distribution
        fallback = True
        for _ in range(NUM_TRIALS):
            pool_idx = torch.multinomial(prob_pool, 1).item()
            candidate_token_id = idx_pool[pool_idx].item()
            if candidate_token_id in small_probs and small_probs[candidate_token_id] >= PROB_THRESHOLD:
                chosen_id = candidate_token_id
                fallback = False
                break

        # Fallback: sample from small model if no acceptance
        if fallback:
            idx_pool = torch.tensor(list(small_probs.keys()))
            prob_pool = torch.tensor(list(small_probs.values()))
            prob_pool = prob_pool / prob_pool.sum()
            pool_idx = torch.multinomial(prob_pool, 1).item()
            chosen_id = idx_pool[pool_idx].item()

        # Get token text and probabilities for the chosen token
        chosen = small_tokenizer.decode([chosen_id])
        large_probability = large_probs.get(chosen_id, 0.0)
        small_probability = small_probs.get(chosen_id, 0.0)

        step_index += 1
        record = dict(
            idx=step_index, 
            text=chosen, 
            token_id=chosen_id,
            fallback=fallback, 
            large_probability=large_probability, 
            small_probability=small_probability,
        )
        yield record

        print(f"{step_index:4d}{'*' if fallback else ' '}\t"
              f"{large_probability:.4f}\t{small_probability:.4f}\t"
              f"{chosen_id}\t'{chosen}'",
              flush=True)

        context += chosen
        if chosen_id == small_tokenizer.eos_token_id:
            break

# ---------------------- high-level convenience -----------------------------
async def run_mixed_decode(prompt: str, max_new_tokens: int = MAX_NEW_TOKENS):
    print("-"*80)
    print("Step\tL_Prob\tS_Prob\tTok_ID\tTok_Txt")
    records = []
    async for record in mixed_decode(prompt, max_new_tokens):
        records.append(record)
    print("-"*80)
    display(html_heat(records))
    fallback_count = sum(record['fallback'] for record in records)
    print(f"Fallback tokens: {fallback_count}/{len(records)} "
          f"({fallback_count/len(records)*100:.2f} %)")

# ------------------------ fire up the engines ------------------------------
await setup_engines()

# --------------------------- example usage ---------------------------------
question = df['question'].iloc[0]

prompt = f"""A conversation between User and Assistant. The User asks a question, and the Assistant responds in two clearly defined sections: 1. Reasoning Process - A step-by-step, logical exploration and analysis of the problem, enclosed within <think> and </think> tags. 2. Answer - A direct and concise response based on the reasoning process, with the final answer enclosed within \\boxed{{}}. For example, 
<think>
reasoning process here
</think>
answer here
\\boxed{{final answer here}}

Now, continue the actual conversation below.
User: {question}
Assistant:
<think>"""

await run_mixed_decode(prompt)

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 83514.90it/s]



CUDA_VISIBLE_DEVICES = 0
torch sees 1 GPU(s)
WARNING 06-03 07:58:28 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7f0797135b10>
WARNING 06-03 07:58:29 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.


Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  14% Completed | 1/7 [00:00<00:01,  4.06it/s]
Loading safetensors checkpoint shards:  29% Completed | 2/7 [00:00<00:02,  2.39it/s]
Loading safetensors checkpoint shards:  43% Completed | 3/7 [00:01<00:01,  2.00it/s]
Loading safetensors checkpoint shards:  57% Completed | 4/7 [00:01<00:01,  1.88it/s]
Loading safetensors checkpoint shards:  71% Completed | 5/7 [00:02<00:01,  1.81it/s]
Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:03<00:00,  1.80it/s]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:03<00:00,  1.82it/s]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:03<00:00,  1.92it/s]




CUDA_VISIBLE_DEVICES = 3
torch sees 1 GPU(s)
WARNING 06-03 07:59:03 [config.py:2972] Casting torch.bfloat16 to torch.float16.
WARNING 06-03 07:59:10 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7f0795db4a00>
WARNING 06-03 07:59:11 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.28it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.24it/s]



Large vocab size: 152064
Small vocab size: 151936
Difference      : 128
Unmatched tokens: 4
--------------------------------------------------------------------------------
Step	L_Prob	S_Prob	Tok_ID	Tok_Txt
  large_output: CompletionOutput(index=0, text='Okay', token_ids=[32313], cumulative_logprob=-0.7150774002075195, logprobs=[{32313: Logprob(logprob=-0.7150774002075195, rank=1, decoded_token='Okay'), 35439: Logprob(logprob=-0.8088274002075195, rank=2, decoded_token='ĠOkay'), 71486: Logprob(logprob=-3.0744524002075195, rank=3, decoded_token='Alright'), 97593: Logprob(logprob=-4.1682024002075195, rank=4, decoded_token='ĠAlright'), 3925: Logprob(logprob=-7.9338274002075195, rank=5, decoded_token='OK'), 6771: Logprob(logprob=-8.32445240020752, rank=6, decoded_token='ĠLet'), 2014: Logprob(logprob=-8.35570240020752, rank=7, decoded_token='ĠTo'), 5512: Logprob(logprob=-8.35570240020752, rank=8, decoded_token='ĠFirst'), 10402: Logprob(logprob=-8.51195240020752, rank=9, decoded_token='ĠOK'),

Fallback tokens: 27/4991 (0.54 %)


In [7]:
# --------------------------- example usage ---------------------------------
question = df['question'].iloc[0]

prompt = f"""A conversation between User and Assistant. The User asks a question, and the Assistant responds in two clearly defined sections: 1. Reasoning Process - A step-by-step, logical exploration and analysis of the problem, enclosed within <think> and </think> tags. 2. Answer - A direct and concise response based on the reasoning process, with the final answer enclosed within \\boxed{{}}. For example, 
<think>
reasoning process here
</think>
answer here
\\boxed{{final answer here}}

Now, continue the actual conversation below.
User: {question}
Assistant:
<think>"""

await run_mixed_decode(prompt)

--------------------------------------------------------------------------------
Step	L_Prob	S_Prob	Tok_ID	Tok_Txt
  large_output: CompletionOutput(index=0, text='Okay', token_ids=[32313], cumulative_logprob=-0.7306316494941711, logprobs=[{32313: Logprob(logprob=-0.7306316494941711, rank=1, decoded_token='Okay'), 35439: Logprob(logprob=-0.7931316494941711, rank=2, decoded_token='ĠOkay'), 71486: Logprob(logprob=-3.0743815898895264, rank=3, decoded_token='Alright'), 97593: Logprob(logprob=-4.1368818283081055, rank=4, decoded_token='ĠAlright'), 3925: Logprob(logprob=-7.9337568283081055, rank=5, decoded_token='OK'), 6771: Logprob(logprob=-8.324381828308105, rank=6, decoded_token='ĠLet'), 5512: Logprob(logprob=-8.340006828308105, rank=7, decoded_token='ĠFirst'), 2014: Logprob(logprob=-8.355631828308105, rank=8, decoded_token='ĠTo'), 10402: Logprob(logprob=-8.496256828308105, rank=9, decoded_token='ĠOK'), 16910: Logprob(logprob=-8.574381828308105, rank=10, decoded_token='Ġokay'), 2055: Logpr

  large_output: CompletionOutput(index=0, text='Okay', token_ids=[32313], cumulative_logprob=-0.22179214656352997, logprobs=[{32313: Logprob(logprob=-0.22179214656352997, rank=1, decoded_token='Okay'), 71486: Logprob(logprob=-1.6280421018600464, rank=2, decoded_token='Alright'), 11578: Logprob(logprob=-6.534292221069336, rank=3, decoded_token='Ok'), 3925: Logprob(logprob=-8.268667221069336, rank=4, decoded_token='OK'), 4416: Logprob(logprob=-8.706167221069336, rank=5, decoded_token='So'), 5338: Logprob(logprob=-8.831167221069336, rank=6, decoded_token='First'), 10061: Logprob(logprob=-9.128042221069336, rank=7, decoded_token='Let'), 1249: Logprob(logprob=-9.346792221069336, rank=8, decoded_token='To'), 11908: Logprob(logprob=-9.628042221069336, rank=9, decoded_token='Oh'), 80022: Logprob(logprob=-10.159292221069336, rank=10, decoded_token='Hmm'), 32462: Logprob(logprob=-10.159292221069336, rank=11, decoded_token='Got'), 39814: Logprob(logprob=-10.409292221069336, rank=12, decoded_token

Fallback tokens: 40/6326 (0.63 %)
